# Lab2 - Hate speech classification - Alicia, Arik, Jonas


# Get existing data

To train the model and get the required files, please first run hate_speech_classification_baseline.ipynb

In [1]:
import os.path
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

if os.path.exists('trained_classifier.pkl'):
    with open('trained_classifier.pkl', 'rb') as fid:
        clf = pickle.load(fid)
else:
    print("No trained classifier found")
    exit()

X = pickle.load(open('X.pkl', 'rb'))
tfidfizer = TfidfVectorizer(max_features=1000)
tfidfizer.fit_transform(X)

<159571x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2842337 stored elements in Compressed Sparse Row format>

# Get Youtube comments

Getting the 100 latest comments from the current top 50 trending videos.

Replace GOOGLE_API_KEY with your Google cloud api key. You can get a key from https://console.developers.google.com/apis/credentials

You also have to activate the Youtube Data API v3 on https://console.developers.google.com/apis/library/youtube.googleapis.com

In [2]:
import csv
import requests

maxResults = '100'
max_trending_videos = '50'
apiKey = 'GOOGLE_API_KEY'
comments = {}
all_comments = []


def get_comments(video_id):
    endpoint = 'https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults={0}&videoId={1}&fields=items(id%2Csnippet%2FtopLevelComment%2Fsnippet%2FtextDisplay)&key={2}'.format(maxResults, video_id, apiKey)
    r = requests.get(endpoint)
    comments = []
    if r.status_code == 200:
        for item in r.json().get('items'):
            comment = {}
            comment['comment_id'] = item.get('id')
            comment['comment'] = item.get('snippet').get('topLevelComment').get('snippet').get('textDisplay')
            comments.append(comment)
    else:
        print(r.json())
    return comments
                   
                
def get_trending_videos():
    endpoint = 'https://www.googleapis.com/youtube/v3/videos?part=snippet&chart=mostPopular&fields=items(id%2Csnippet%2Ftitle%2Csnippet%2FchannelId%2Csnippet%2FchannelTitle)&maxResults={0}&key={1}'.format(max_trending_videos, apiKey)
    r = requests.get(endpoint)
    items = r.json().get('items')
    
    for item in items:
        video_id = item.get('id')
        title = item.get('snippet').get('title')
        comments = get_comments(video_id)
        if comments:
            for comment in comments:
                comment_tuple = (video_id, title, comment['comment_id'], comment['comment'])
                all_comments.append(comment_tuple)
            
    
def write_csv():
    with open('comments.csv', mode='w') as comments_file:
        comments_writer = csv.writer(comments_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        comments_writer.writerow(['ID', 'VIDEOID', 'COMMENT', 'VIDEONAME'])
        
        for comment in all_comments:
            video_id = comment[0]
            video_name = comment[1]
            id = comment[2]
            comment = comment[3].replace('<br />', ' ').replace('<b>', ' ').encode('ascii', 'ignore').decode('ascii')
            comments_writer.writerow([id, video_id, comment, video_name])


print('Getting comments from Youtube...')
get_trending_videos()
write_csv()
print('finished writing csv')


Getting comments from Youtube...
finished writing csv


In [6]:
import pandas as pd

df = pd.read_csv('comments.csv')
df.columns = ['ID', 'VIDEOID', 'COMMENT', 'VIDEONAME']
    
video_ids = df['VIDEOID'].drop_duplicates()

comments = df['COMMENT']

# Test on new data

In [7]:
result = {}

for videoId in video_ids:
    result[videoId] = {'hate_comment_count': 0, 'total_comment_count': 0, 'video_name': ''}

In [8]:
tf_idf_matrix_comments = tfidfizer.transform(comments.astype('U'))
comments_result = clf.predict(tf_idf_matrix_comments)

In [14]:
from IPython.display import display, Markdown

cnt = 0
for index, row in df.iterrows():
    cnt += 1
    is_hate_speech = comments_result[index]
    current_video_id = row['VIDEOID']
    
    result_by_video_id = result.get(current_video_id);
    
    if result_by_video_id is not None:

        if is_hate_speech == 1:
            item = '* Video id: ' + row['VIDEOID'] + '<br> Is hate speech: ' + row['COMMENT']
            result_by_video_id['hate_comment_count'] += 1
            display(Markdown(item))

        result_by_video_id['total_comment_count'] += 1
        
        result_by_video_id['video_name'] = row['VIDEONAME']

* Video id: QaZ93sibpk0<br> Is hate speech: OH MY GOD FLASHING LIGHTS AT THE BEGINNING AND KANYE OPENING THE DOOR LITERAL CHILLS GUYS

* Video id: QaZ93sibpk0<br> Is hate speech: Why did she agree to do this if she doesnt like answering questions? 

* Video id: QaZ93sibpk0<br> Is hate speech: Camera Guy: Whats the most over used phrase?   Kim: Its Lit   Everyone Else: Yeah aIght!

* Video id: QaZ93sibpk0<br> Is hate speech:  Kanye is a mad man 

* Video id: QaZ93sibpk0<br> Is hate speech: I usually hate these segments because the interviewee is so unnatural and scripted. Kim seemed so natural and comfortable

...

* Video id: C4-oqJc_Q_A<br> Is hate speech: Nein nein ich Weine nicht, mir ist nur was ins Auge gekommen.  Danke dir Dirk, du hast die NBA verndert und warst immer ein Vorbild.  Gott segne dich

* Video id: C4-oqJc_Q_A<br> Is hate speech: Only reason why i really like Mavericks basketball is that they have a Dirk.

* Video id: C4-oqJc_Q_A<br> Is hate speech: Damn even Larry legend was there. That guy still has the goat aura in him you can tell and the classy fans of Dallas know as well. But Im so happy for dirk and what he brought to nba

* Video id: C4-oqJc_Q_A<br> Is hate speech: Great send off!  Mark cuban looked like he was bout to cry lol

* Video id: C4-oqJc_Q_A<br> Is hate speech: Only messed up thing is that rap song at the end. Play some real rap like Bone lol

In [10]:
most_hate_comment_count_video_id= ''
most_hate_comment_count = 0;

for key, value in result.items():
    if (value.get('hate_comment_count') > most_hate_comment_count):
        most_hate_comment_count = value.get('hate_comment_count')
        most_hate_comment_count_video_id = key
        
direct_url = 'https://www.youtube.com/watch?v=' + most_hate_comment_count_video_id

In [11]:
print('Most hated trending video on youtube: ')
print(direct_url)

Most hated trending video on youtube: 
https://www.youtube.com/watch?v=FZIKVN9BQQA


In [12]:
from IPython.display import HTML
embed_url = 'https://www.youtube.com/embed/' + most_hate_comment_count_video_id
HTML('<iframe width="560" height="315" src="{0}" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'.format(embed_url))